In [1]:
import torch
import tensorly as tl
from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tensorly.decomposition import tensor_train
from tensorly.decomposition import tensor_train
from tensorly import tt_to_tensor
from tensorly.decomposition import matrix_product_state
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import time
import os
import copy

import torch.nn.init as init

print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())


PyTorch Version: 1.13.1
Torchvision Version: 0.14.1
GPU is available? True


In [2]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Convert to tensor and scale to [0, 1]
ts = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0,), (1,))])
mnist_trainset = datasets.MNIST('/home/c/cl237/', train=True, download=True, transform=ts)
mnist_testset = datasets.MNIST(root='/home/c/cl237/', train=False, download=True, transform=ts)

In [3]:
from torch.utils.data import Subset
train_subset_size = int(0.1 * len(mnist_trainset))
test_subset_size = int(0.0999 * len(mnist_testset))
train_indices = list(range(len(mnist_trainset)))
test_indices = list(range(len(mnist_testset)))
torch.manual_seed(10)

seed_value = 10
np.random.seed(seed_value)

np.random.shuffle(train_indices)
np.random.shuffle(test_indices)

train_subset_indices = train_indices[:train_subset_size]
test_subset_indices = test_indices[:test_subset_size]

mnist_trainset = Subset(mnist_trainset, train_subset_indices)
mnist_testset = Subset(mnist_testset, test_subset_indices)

In [4]:
x_d0 = mnist_trainset[0][0].size()[0]
x_d1 = mnist_trainset[0][0].size()[1]
x_d2 = mnist_trainset[0][0].size()[2]
# ([1, 28, 28])
N = x_d3 = len(mnist_trainset)
K = 10
x_train = torch.empty((N,x_d0*x_d1*x_d2), device=device)
# (60000, 28*28)

y_train = torch.empty(N, dtype=torch.long)

#torch.empty() return a tensor filled with garbage values Dimention: N by x_d0*x_d1*x_d2

for i in range(N):
     x_train[i,:] = torch.reshape(mnist_trainset[i][0], (1, x_d0*x_d1*x_d2))
     y_train[i] = mnist_trainset[i][1]
x_train = torch.t(x_train)
y_one_hot = torch.zeros(N, K).scatter_(1, torch.reshape(y_train, (N, 1)), 1)
y_one_hot = torch.t(y_one_hot).to(device=device)
y_train = y_train.to(device=device)

# Manipulate test set
N_test = x_d3_test = len(mnist_testset)
x_test = torch.empty((N_test,x_d0*x_d1*x_d2), device=device)
y_test = torch.empty(N_test, dtype=torch.long)
for i in range(N_test):
     x_test[i,:] = torch.reshape(mnist_testset[i][0], (1, x_d0*x_d1*x_d2))
     y_test[i] = mnist_testset[i][1]
x_test = torch.t(x_test)
y_test_one_hot = torch.zeros(N_test, K).scatter_(1, torch.reshape(y_test, (N_test, 1)), 1)
y_test_one_hot = torch.t(y_test_one_hot).to(device=device)
y_test = y_test.to(device=device)

In [5]:
#### Reshape X to X-bar
x_trainTensor = torch.reshape(x_train, (x_d1, x_d2, x_d0,-1))
x_testTensor  = torch.reshape(x_test, (x_d1, x_d2, x_d0,-1))
print(x_trainTensor.shape)
print(x_testTensor.shape)

torch.Size([28, 28, 1, 6000])
torch.Size([28, 28, 1, 999])


In [6]:
#### Chenyang: you need to write a function to transform the x_trainTensor, x_testTensor to (H'W')*(l^2C)*n
#### follow the paper in https://arxiv.org/pdf/1611.03214.pdf   this paper donot have the stride, we should also include stride
#### how to make it faster;  make sure the code is correct?
def inputX_CNN(x_Tensor, filter_size, stride):
    H, W, C, n = x_Tensor.size()
    Hprime = torch.floor(torch.tensor((H-filter_size)/stride))+1
    Hprime = Hprime.to(torch.int)
    Wprime = torch.floor(torch.tensor((W-filter_size)/stride))+1
    Wprime = Wprime.to(torch.int)
    Xtranform = torch.zeros((Hprime * Wprime, filter_size * filter_size * C, n), device=device)
    for i in range(n):
      Data =  x_Tensor[:,:,:,i]
      for hh in range(Hprime):
        for ww in range(Wprime):
          #  print(range(ww * stride, ww * stride + filter_size))
            DataTemp = Data[range(hh * stride, hh * stride + filter_size), :,:]
            DataTemp = DataTemp[:, range(ww * stride, ww * stride + filter_size), :]
            Xtranform[Hprime * hh + ww, :, i] = torch.reshape(DataTemp, (1, filter_size * filter_size * C))
    return Xtranform

In [7]:
### from X-bar to X-bar-bar
filter_size=3
stride = 2
x_trainTS = inputX_CNN(x_trainTensor,filter_size,stride)
x_testTS  = inputX_CNN(x_testTensor,filter_size,stride)

In [8]:
#### To-do-list
#### Chenyang: you need to write a function to update W for CNN
def updateWb_CNN(U, V, W, W_tensor_rec, alpha, rho,tau): # W:previous W
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)  #w shape need to take a look at
    d,N = V.size()
    I = torch.eye(N, device=device)
    U_prime = torch.t(U).reshape(n*Hprime*Wprime,-1)
    # _, col_U = U_prime.size()
    Wstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(V),V))+(alpha+tau)*I), rho*torch.mm(torch.t(V),U_prime)+alpha*W+tau*W_tensor2matrix)
    # bstar = 0
    return Wstar
# def updateWb_CNN(U, V, W, W_tensor_rec, alpha, rho,tau): # W:previous W, v for the first layer should be X-bar-bar-prime
#     W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
#     W_tensor2matrix = W_tensor_rec.reshape(W.shape)  #w shape need to take a look at
#     d,N = V.size()
#     I = torch.eye(N, device=device)
#     U_prime = torch.t(U).reshape(n*Hprime*Wprime,-1)
#     A = rho*torch.mm(torch.t(V),U_prime)+alpha*W+tau*W_tensor2matrix
#     A = torch.t(A)
#     # _, col_U = U_prime.size()
#     Wstar_transpose = torch.mm(A, torch.inverse(rho*(torch.mm(torch.t(V), V))+(alpha+tau)*I))
#     Wstar =  torch.t(Wstar_transpose)
#     return Wstar

def updateWb_CNNorg(U, V, W, alpha, rho):
    d,N = V.size()
    I = torch.eye(N, device=device)
    #_, col_U = U.size()
    U_prime = torch.t(U).reshape(n*Hprime*Wprime,-1)
    Wstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(V),V))+alpha*I), rho*torch.mm(torch.t(V),U_prime)+alpha*W)
    
    return Wstar

def updateV(U1,U2,W,b,rho,gamma):
    _, d = W.size()
    I = torch.eye(d, device=device) #Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(W),W))+gamma*I), rho*torch.mm(torch.t(W),U2-b.repeat(1,col_U2))+gamma*U1)
    # check how to get V* W*
    # torch.mm*=(input,mat2) performs a matrix multiplication of the matrices input and mat2.
    #   if input is a (n*m) tensor,mat2 is a (m*p) tensor, output will be (n*p) tensor
    return Vstar

def updateWb_org(U, V, W, b, alpha, rho):
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse(alpha*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

def updateWb(U, V, W, b, W_tensor_rec, alpha, rho,tau):
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

# U update  E(1) in global convergent
def relu_prox(a, b, gamma, d, N):
    val = torch.empty(d,N, device=device)
    x = (a+gamma*b)/(1+gamma)
    y = torch.min(b,torch.zeros(d,N, device=device))

    val = torch.where(a+gamma*b < 0, y, torch.zeros(d,N, device=device))
    val = torch.where(((a+gamma*b >= 0) & (b >=0)) | ((a*(gamma-np.sqrt(gamma*(gamma+1))) <= gamma*b) & (b < 0)), x, val)
    val = torch.where((-a <= gamma*b) & (gamma*b <= a*(gamma-np.sqrt(gamma*(gamma+1)))), b, val)
    return val

In [9]:
########### parameter setup
#df = pd.DataFrame()
#df.to_csv('C:/Users/Mark/Desktop/CNNmnist_hyperparameter_test.csv')
niter = 2000
rank = 220
tau = 0.2
gamma = 0.5
alpha = 1
rho = 0.5

print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)

loss1 = np.empty(niter)
loss2 = np.empty(niter)
accuracy_train = np.empty(niter)
accuracy_test = np.empty(niter)
time1 = np.empty(niter)

results = torch.zeros(1, 5, niter)

S = 32 ### number of filters 2^5
H, W, C, n = x_trainTensor.size()   # n is the same thing as N
Hprime = torch.floor(torch.tensor((H-filter_size)/stride))+1
Hprime = Hprime.to(torch.int)
Wprime = torch.floor(torch.tensor((W-filter_size)/stride))+1
Wprime = Wprime.to(torch.int)



for Out_iter in range(1):
    rank_initial = 700
    seed = 10 + 10*Out_iter
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)

    ## Chenyang
    d1 =  Hprime * Wprime * S
    d2 =  1024
    d3 =  1024
    d4 =  10


    W1 = 0.01*torch.randn(filter_size * filter_size * C, S, device=device)  ## How people usually initialize the CNN kernel?
    #W1_torch_tensor = W1.reshape((filter_size, filter_size, C, 2,2,2,2,2)) ## TBD
    #W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
    #factors1 = tensor_train(W1_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, 1))
    #W1_tl_tensor_rec = tt_to_tensor(factors1)
    b1 = 0*torch.ones(d1, 1, device=device) # 0 is stable


    W2 = init.uniform_(torch.empty(d2, d1, device=device), a=-0.02, b=0.02)
    W2_torch_tensor = W2.reshape((2*Hprime,2*Wprime,4,4,4,4,4,2,2,2)) # 2^10 and Hprime * Wprime  *2^5*1^3
    W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
    factors2 = tensor_train(W2_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    # we have 9 rank_initial (r1,...,r9) here plus 2 of 1s (r0 & r10).
    # factors: set of tensor cores
    # http://tensorly.org/stable/user_guide/tensor_decomposition.html
    # http://tensorly.org/stable/modules/generated/tensorly.decomposition.tensor_train.html#tensorly.decomposition.tensor_train
    W2_tl_tensor_rec = tt_to_tensor(factors2)
    # tt_to_tensor(factors):Re-assembles ‘factors’, which represent a tensor in TT format into the corresponding full tensor
    #      facros: list of 3d-arrays tt-cores           output_tensor: ndarray
    b2 = 0*torch.ones(d2, 1, device=device)


    W3 = init.uniform_(torch.empty(d3, d2, device=device), a=-0.02, b=0.02)
    W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
    factors3 = tensor_train(W3_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W3_tl_tensor_rec = tt_to_tensor(factors3)
    b3 = 0*torch.ones(d3, 1, device=device)

    W4 = init.uniform_(torch.empty(d4, d3, device=device), a=-0.02, b=0.02)
    b4 = 0*torch.ones(d4, 1, device=device)

    ## Chenyang: this is for the initialization of U1
    x_trainTS1 = torch.reshape(x_trainTS, (-1,n))
    x_trainTS1 = torch.t(x_trainTS1)
    x_trainTS1 = torch.reshape(x_trainTS1, (-1, filter_size * filter_size * C))   ### this is X-bar-bar'
    U1prime = torch.matmul(x_trainTS1, W1)
    U1prime = torch.reshape(U1prime, (n,-1))
    U1 = torch.t(U1prime)

    V1 = nn.ReLU()(U1)
    U2 = torch.addmm(b2.repeat(1, N), W2, V1)
    V2 = nn.ReLU()(U2)
    U3 = torch.addmm(b3.repeat(1, N), W3, V2)
    V3 = nn.ReLU()(U3)
    U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    V4 = U4

    ### to create the X-bar-bar' for test data
    x_testTS1 = torch.reshape(x_testTS, (-1, N_test))
    x_testTS1 = torch.t(x_testTS1)
    x_testTS1 = torch.reshape(x_testTS1, (-1, filter_size * filter_size * C))   ### this is X-bar-bar' test


    # Iterations
    print('Train on', N, 'samples, validate on', N_test, 'samples')
    for k in range(niter):
        start = time.time()

  # update for last layer
        # update V4
        V4 = (y_one_hot + gamma*U4 + alpha*V4)/(1 + gamma + alpha)

        # update U4
        U4 = (gamma*V4 + rho*(torch.mm(W4,V3) + b4.repeat(1,N)))/(gamma + rho)

        # update W4 and b4
        W4, b4 = updateWb_org(U4,V3,W4,b4,alpha,rho)
        # W3_torch_tensor = W3.reshape((d3,4,4,4,4,4,4))
        # W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
        # factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,1))
        # W3_tl_tensor_rec = tt_to_tensor(factors3)


  # update for 3nd layer
        # update V3
        V3 = updateV(U3,U4,W4,b4,rho,gamma)

        # update U3
        U3 = relu_prox(V3,(rho*torch.addmm(b3.repeat(1,N), W3, V2) + alpha*U3)/(rho + alpha),(rho + alpha)/gamma,d3,N)

        # update W3 and b3
        W3, b3 = updateWb(U3,V2,W3,b3,W3_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)
        W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4))
        W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W3_tl_tensor_rec = tt_to_tensor(factors3)


  # update for 2nd layer
        # update V2
        V2 = updateV(U2,U3,W3,b3,rho,gamma)

        # update U2
        U2 = relu_prox(V2,(rho*torch.addmm(b2.repeat(1,N), W2, V1) + alpha*U2)/(rho + alpha),(rho + alpha)/gamma,d2,N)

        # update W2 and b2
        W2, b2 = updateWb(U2,V1,W2,b2,W2_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)
        W2_torch_tensor = W2.reshape((2*Hprime,2*Wprime,4,4,4,4,4,2,2,2))
        W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
        factors2 = tensor_train(W2_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W2_tl_tensor_rec = tt_to_tensor(factors2)


 # update for 1st layer
        # update V1
        V1 = updateV(U1,U2,W2,b2,rho,gamma)

        ## Chenyang
        XprimeW = torch.reshape(torch.matmul(x_trainTS1, W1), (n,-1))
        XprimeWtranspose = torch.t(XprimeW)
        # update U1
        U1 = relu_prox(V1,(rho*XprimeWtranspose + alpha*U1)/(rho + alpha),(rho + alpha)/gamma,d1,N)

        # update W1 and b1
        ## Chenyang: you need to use the new function updateWb_CNN
        W1 = updateWb_CNNorg(U1,x_trainTS1,W1,alpha,rho)

        # G update
        #W1_torch_tensor = W1.reshape((filter_size, filter_size, C, 2,2,2,2,2))
        #W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
        #factors1 = tensor_train(W1_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,1))
        #W1_tl_tensor_rec = tt_to_tensor(factors1)




  # prediction for trainning data
        ### we should use reconstructed tensnor W1 for prediction
        XprimeW = torch.reshape(torch.matmul(x_trainTS1, torch.as_tensor(W1,device=device).reshape((filter_size * filter_size * C, S)).float()), (n,-1))
        XprimeWtranspose = torch.t(XprimeW)
        a1_train = nn.ReLU()(XprimeWtranspose)
        #a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_train))
        #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        pred = torch.argmax(torch.addmm(b4.repeat(1, N), W4, a3_train), dim=0)
        # check argmax and addmm, dim=0

  #Prediction for test data
        XprimeWtest = torch.reshape(torch.matmul(x_testTS1, torch.as_tensor(W1,device=device).reshape((filter_size * filter_size * C, S)).float()), (N_test,-1))
        XprimeWtesttranspose = torch.t(XprimeWtest)
        a1_test = nn.ReLU()(XprimeWtesttranspose) ## Chenyang
        #a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_test))
        pred_test = torch.argmax(torch.addmm(b4.repeat(1, N_test), W4, a3_test), dim=0)


    #emperical loss
        loss1[k] = gamma/2*torch.pow(torch.dist(V4,y_one_hot,2),2).cpu().numpy()

        # Eq (5) in paper
        ## Chenyang
        loss2[k] = loss1[k] + rho/2*torch.pow(torch.dist(XprimeWtranspose,U1,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b4.repeat(1,N), W4, V3),U4,2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V2,nn.ReLU()(U2),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V3,nn.ReLU()(U3),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V4,U4,2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W2.reshape((2*Hprime,2*Wprime,4,4,4,4,4,2,2,2)),torch.as_tensor(W2_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W3.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        # +tau/2*torch.pow(torch.dist(W3.reshape((d3,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy()
        # +tau/2*torch.pow(torch.dist(W4.reshape((50,30,K)),torch.as_tensor(W4_tl_tensor_rec,device=device),2),2).cpu().numpy()

        # compute training accuracy
        correct_train = pred == y_train
        accuracy_train[k] = np.mean(correct_train.cpu().numpy())

        # compute validation accuracy
        correct_test = pred_test == y_test
        accuracy_test[k] = np.mean(correct_test.cpu().numpy())

        # compute training time
        stop = time.time()
        duration = stop - start
        time1[k] = duration

        # print results
        print('Repeatition', Out_iter + 1, 'Epoch', k + 1, '/', niter, '\n',
              '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k],
              '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k])

##############
############## compute CR
    # factors1_shape=[f.shape for f in factors1]
    # Sum_of_variables_factors1=sum(list(x*y*z for x,y,z in factors1_shape))
    factors2_shape=[f.shape for f in factors2]
    Sum_of_variables_factors2=sum(list(x*y*z for x,y,z in factors2_shape))
    factors3_shape=[f.shape for f in factors3]
    Sum_of_variables_factors3=sum(list(x*y*z for x,y,z in factors3_shape))
    total_variabels=Sum_of_variables_factors2+Sum_of_variables_factors3

    layer2_CR = Sum_of_variables_factors2/(d1*d2).item()
    layer3_CR = Sum_of_variables_factors3/(d2*d3)
    Compressedlayers_CR = total_variabels/(d1*d2+d2*d3).item()
    Compressedlayers_CR2 = (total_variabels+d3*d4)/(d1*d2+d2*d3+d3*d4).item()

    # print("CR_1 = "+str(CR_1)+"   "+"CR_2 = " +str(CR_2))


    results[Out_iter,0,:] = torch.tensor(loss1)
    results[Out_iter,1,:] = torch.tensor(loss2)
    results[Out_iter,2,:] = torch.tensor(accuracy_train)
    results[Out_iter,3,:] = torch.tensor(accuracy_test)
    results[Out_iter,4,:] = torch.tensor(time1)
    CR=(layer2_CR,layer3_CR,Compressedlayers_CR,Compressedlayers_CR2)

    #this postion to add new row into existing table
    #df=pd.read_csv('C:/Users/Mark/Desktop/CNNmnist_hyperparameter_test.csv')
    #new_row = {'rank':rank, 'tau':tau, 'gamma':gamma, 'rho':rho, 'alpha':alpha, \
    #           'loss1':loss1[niter-1], 'loss2':loss2[niter-1], 'accuracy_train':accuracy_train[niter-1], \
    #           'accuracy_test':accuracy_test[niter-1], 'time':time1[niter-1], 'layer2_CR':layer2_CR, \
    #          'layer3_CR':layer3_CR, 'Compressedlayers_CR':Compressedlayers_CR}
    #df=df.append(new_row,ignore_index=True)
    #df.to_csv('C:/Users/Mark/Desktop/CNNmnist_hyperparameter_test.csv',index=False)


filename="UniformScaled"+ "niter_"+ str(niter) + "rank_" + str(rank) + "tau_" + str(tau) + "gamma_" + str(gamma) + \
"rho_" + str(rho) + "alpha_" + str(alpha) + ".mat"
from scipy.io import savemat
%cd /home/c/cl237/TenBCDCNN_mnist_init
savemat (filename, {'results': torch.Tensor.numpy(results), 'Compression Ratio':CR})


rank= 220 tau= 0.2 gamma= 0.5 rho= 0.5 alpha 1
Train on 6000 samples, validate on 999 samples
Repeatition 1 Epoch 1 / 2000 
 - time: 5.732807636260986 - sq_loss: 540.1578979492188 - tot_loss: 701.4925472422095 - acc: 0.44366666666666665 - val_acc: 0.42742742742742745
Repeatition 1 Epoch 2 / 2000 
 - time: 5.4185380935668945 - sq_loss: 240.0701446533203 - tot_loss: 404.02175382913555 - acc: 0.558 - val_acc: 0.5485485485485485
Repeatition 1 Epoch 3 / 2000 
 - time: 5.474583148956299 - sq_loss: 129.71270751953125 - tot_loss: 231.60392570169643 - acc: 0.606 - val_acc: 0.5995995995995996
Repeatition 1 Epoch 4 / 2000 
 - time: 5.433317184448242 - sq_loss: 70.68147277832031 - tot_loss: 134.1674422778189 - acc: 0.6406666666666667 - val_acc: 0.6386386386386387
Repeatition 1 Epoch 5 / 2000 
 - time: 5.4342405796051025 - sq_loss: 38.45220184326172 - tot_loss: 79.46742787119001 - acc: 0.6743333333333333 - val_acc: 0.6706706706706707
Repeatition 1 Epoch 6 / 2000 
 - time: 5.454541444778442 - sq_los

Repeatition 1 Epoch 49 / 2000 
 - time: 5.398761987686157 - sq_loss: 0.000771883933339268 - tot_loss: 0.500740650726948 - acc: 0.8978333333333334 - val_acc: 0.8968968968968969
Repeatition 1 Epoch 50 / 2000 
 - time: 5.397097587585449 - sq_loss: 0.0007244901498779655 - tot_loss: 0.486830385052599 - acc: 0.8995 - val_acc: 0.8958958958958959
Repeatition 1 Epoch 51 / 2000 
 - time: 5.3898606300354 - sq_loss: 0.0006804834119975567 - tot_loss: 0.47366133546456696 - acc: 0.9001666666666667 - val_acc: 0.8948948948948949
Repeatition 1 Epoch 52 / 2000 
 - time: 5.351000547409058 - sq_loss: 0.0006395326927304268 - tot_loss: 0.46118032264057546 - acc: 0.9011666666666667 - val_acc: 0.8948948948948949
Repeatition 1 Epoch 53 / 2000 
 - time: 5.422040224075317 - sq_loss: 0.0006014843820594251 - tot_loss: 0.4493405204149894 - acc: 0.9011666666666667 - val_acc: 0.8968968968968969
Repeatition 1 Epoch 54 / 2000 
 - time: 5.38813328742981 - sq_loss: 0.0005660926108248532 - tot_loss: 0.4381006866809912 - ac

Repeatition 1 Epoch 97 / 2000 
 - time: 5.372770309448242 - sq_loss: 8.923280256567523e-05 - tot_loss: 0.21778062406083337 - acc: 0.9281666666666667 - val_acc: 0.924924924924925
Repeatition 1 Epoch 98 / 2000 
 - time: 5.3907153606414795 - sq_loss: 8.703320054337382e-05 - tot_loss: 0.21540931784547865 - acc: 0.9286666666666666 - val_acc: 0.9259259259259259
Repeatition 1 Epoch 99 / 2000 
 - time: 5.40606951713562 - sq_loss: 8.494553185300902e-05 - tot_loss: 0.21309262788417982 - acc: 0.929 - val_acc: 0.9259259259259259
Repeatition 1 Epoch 100 / 2000 
 - time: 5.390386343002319 - sq_loss: 8.296033774968237e-05 - tot_loss: 0.2108289450785378 - acc: 0.9296666666666666 - val_acc: 0.9259259259259259
Repeatition 1 Epoch 101 / 2000 
 - time: 5.393552303314209 - sq_loss: 8.106041786959395e-05 - tot_loss: 0.20861644602700835 - acc: 0.9301666666666667 - val_acc: 0.9269269269269269
Repeatition 1 Epoch 102 / 2000 
 - time: 5.391930341720581 - sq_loss: 7.925630052341148e-05 - tot_loss: 0.206452872288

Repeatition 1 Epoch 144 / 2000 
 - time: 5.375225782394409 - sq_loss: 4.360059028840624e-05 - tot_loss: 0.14465107754731435 - acc: 0.9436666666666667 - val_acc: 0.9379379379379379
Repeatition 1 Epoch 145 / 2000 
 - time: 5.426476955413818 - sq_loss: 4.318566061556339e-05 - tot_loss: 0.14364122745464555 - acc: 0.9438333333333333 - val_acc: 0.9379379379379379
Repeatition 1 Epoch 146 / 2000 
 - time: 5.39169454574585 - sq_loss: 4.277794505469501e-05 - tot_loss: 0.14264592834806536 - acc: 0.9441666666666667 - val_acc: 0.9379379379379379
Repeatition 1 Epoch 147 / 2000 
 - time: 5.3786351680755615 - sq_loss: 4.2386895074741915e-05 - tot_loss: 0.1416651571787952 - acc: 0.9445 - val_acc: 0.9379379379379379
Repeatition 1 Epoch 148 / 2000 
 - time: 5.357417106628418 - sq_loss: 4.2006846342701465e-05 - tot_loss: 0.14069851154199567 - acc: 0.9448333333333333 - val_acc: 0.9379379379379379
Repeatition 1 Epoch 149 / 2000 
 - time: 5.368403434753418 - sq_loss: 4.162788172834553e-05 - tot_loss: 0.13974

Repeatition 1 Epoch 191 / 2000 
 - time: 5.37646746635437 - sq_loss: 3.0586896173190325e-05 - tot_loss: 0.10917820335307625 - acc: 0.9543333333333334 - val_acc: 0.94994994994995
Repeatition 1 Epoch 192 / 2000 
 - time: 5.413650989532471 - sq_loss: 3.039884177269414e-05 - tot_loss: 0.10862331975367852 - acc: 0.9546666666666667 - val_acc: 0.94994994994995
Repeatition 1 Epoch 193 / 2000 
 - time: 5.386308431625366 - sq_loss: 3.0210576369427145e-05 - tot_loss: 0.1080746150953928 - acc: 0.955 - val_acc: 0.94994994994995
Repeatition 1 Epoch 194 / 2000 
 - time: 5.3836822509765625 - sq_loss: 3.0027300454094075e-05 - tot_loss: 0.10753188956987288 - acc: 0.955 - val_acc: 0.94994994994995
Repeatition 1 Epoch 195 / 2000 
 - time: 5.378396987915039 - sq_loss: 2.9846230972907506e-05 - tot_loss: 0.1069951343433786 - acc: 0.9551666666666667 - val_acc: 0.94994994994995
Repeatition 1 Epoch 196 / 2000 
 - time: 5.371246814727783 - sq_loss: 2.966712781926617e-05 - tot_loss: 0.10646421509300126 - acc: 0.9

...

Repeatition 1 Epoch 1792 / 2000 
 - time: 5.434890508651733 - sq_loss: 2.883588422264438e-06 - tot_loss: 0.01573795950080239 - acc: 0.9918333333333333 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1793 / 2000 
 - time: 5.383991241455078 - sq_loss: 2.881660066123004e-06 - tot_loss: 0.015730021672288784 - acc: 0.9918333333333333 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1794 / 2000 
 - time: 5.3936522006988525 - sq_loss: 2.8796816877729725e-06 - tot_loss: 0.015722086936420965 - acc: 0.9918333333333333 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1795 / 2000 
 - time: 5.368381500244141 - sq_loss: 2.8781632863683626e-06 - tot_loss: 0.015714166047473554 - acc: 0.9918333333333333 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1796 / 2000 
 - time: 5.393205642700195 - sq_loss: 2.8768565698555904e-06 - tot_loss: 0.01570624418714033 - acc: 0.9918333333333333 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1797 / 2000 
 - time: 5.4114930629730225 - sq_loss: 2.8746842417604057e-0

Repeatition 1 Epoch 1838 / 2000 
 - time: 5.394270420074463 - sq_loss: 2.8039562494086567e-06 - tot_loss: 0.015380292519330397 - acc: 0.9918333333333333 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1839 / 2000 
 - time: 5.38583517074585 - sq_loss: 2.8020017452945467e-06 - tot_loss: 0.015372691841821506 - acc: 0.9918333333333333 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1840 / 2000 
 - time: 5.387651681900024 - sq_loss: 2.8000285965390503e-06 - tot_loss: 0.015365091101830331 - acc: 0.9918333333333333 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1841 / 2000 
 - time: 5.3833394050598145 - sq_loss: 2.798348305077525e-06 - tot_loss: 0.015357505700831098 - acc: 0.9918333333333333 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1842 / 2000 
 - time: 5.380490779876709 - sq_loss: 2.7964674700342584e-06 - tot_loss: 0.015349932989010995 - acc: 0.9918333333333333 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1843 / 2000 
 - time: 5.372605085372925 - sq_loss: 2.794301508401986e-0

Repeatition 1 Epoch 1884 / 2000 
 - time: 5.38494873046875 - sq_loss: 2.726109642026131e-06 - tot_loss: 0.015038217043047553 - acc: 0.9918333333333333 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1885 / 2000 
 - time: 5.390311002731323 - sq_loss: 2.7239100290898932e-06 - tot_loss: 0.01503095635639511 - acc: 0.9918333333333333 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1886 / 2000 
 - time: 5.395016431808472 - sq_loss: 2.7215669433644507e-06 - tot_loss: 0.015023707413638477 - acc: 0.9918333333333333 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1887 / 2000 
 - time: 5.4069459438323975 - sq_loss: 2.7197484087082557e-06 - tot_loss: 0.015016458381705887 - acc: 0.9918333333333333 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1888 / 2000 
 - time: 5.41319465637207 - sq_loss: 2.7187099931325065e-06 - tot_loss: 0.015009214265728588 - acc: 0.9918333333333333 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1889 / 2000 
 - time: 5.375641107559204 - sq_loss: 2.717382585615269e-06 

Repeatition 1 Epoch 1930 / 2000 
 - time: 5.393530607223511 - sq_loss: 2.6509746930969413e-06 - tot_loss: 0.01471132615361057 - acc: 0.9918333333333333 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1931 / 2000 
 - time: 5.342653274536133 - sq_loss: 2.6489326501177857e-06 - tot_loss: 0.014704378459964572 - acc: 0.9918333333333333 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1932 / 2000 
 - time: 5.40298056602478 - sq_loss: 2.6475747745280387e-06 - tot_loss: 0.014697431845934262 - acc: 0.9918333333333333 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1933 / 2000 
 - time: 5.380552768707275 - sq_loss: 2.646674147399608e-06 - tot_loss: 0.01469049430306768 - acc: 0.9918333333333333 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1934 / 2000 
 - time: 5.428758859634399 - sq_loss: 2.6456659725226928e-06 - tot_loss: 0.014683555702049489 - acc: 0.9918333333333333 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1935 / 2000 
 - time: 5.391937494277954 - sq_loss: 2.6442403395776637e-06 

Repeatition 1 Epoch 1977 / 2000 
 - time: 5.376485347747803 - sq_loss: 2.5794722660066327e-06 - tot_loss: 0.014391588198168393 - acc: 0.992 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1978 / 2000 
 - time: 5.390379190444946 - sq_loss: 2.577725126684527e-06 - tot_loss: 0.014384925684817062 - acc: 0.992 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1979 / 2000 
 - time: 5.361340761184692 - sq_loss: 2.576360657258192e-06 - tot_loss: 0.014378280409619037 - acc: 0.992 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1980 / 2000 
 - time: 5.37030816078186 - sq_loss: 2.5747101517481497e-06 - tot_loss: 0.014371647853340618 - acc: 0.992 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1981 / 2000 
 - time: 5.387356519699097 - sq_loss: 2.5739980173966615e-06 - tot_loss: 0.014365014823715683 - acc: 0.992 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1982 / 2000 
 - time: 5.3963940143585205 - sq_loss: 2.5727140382514335e-06 - tot_loss: 0.01435838964798677 - acc: 0.992 - val_acc: 0.98098